In [ ]:
!nvidia-smi

Mon May  2 13:00:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    36W / 250W |    971MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **We train on GPU and test on CPU for comparison with quantization** 

Fistly, we train Teacher and noKD

In [ ]:
!python train.py

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Layer (type:depth-idx)                   Output Shape              Param #
DistilLog                                [50, 2]                   --
├─GRU: 1-1                               [50, 50, 128]             160,512
├─Linear: 1-2                            [50, 2]                   258
Total params: 160,770
Trainable params: 160,770
Non-trainable params: 0
Total mult-adds (M): 401.29
Input size (MB): 0.30
Forward/backward pass size (MB): 2.56
Params size (MB): 0.64
Estimated Total Size (MB): 3.50
Train Epoch: 1/300 [12000/12000 (100%)]  Loss: 134.978974: 100% 240/240 [00:01<00:00, 158.65it/s]
Train Epoch: 2/300 [12000/12000 (100%)]  Loss: 106.857896: 100% 240/240 [00:01<00:00, 158.73it/s]
Train Epoch: 3/

Then, student model learn from teacher model

In [ ]:
!python teach.py

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Epoch 1: 100% 240/240 [00:01<00:00, 162.56it/s]
Loss:-0.56
Epoch 2: 100% 240/240 [00:01<00:00, 163.15it/s]
Loss:-0.72
Epoch 3: 100% 240/240 [00:01<00:00, 165.05it/s]
Loss:-0.76
Epoch 4: 100% 240/240 [00:01<00:00, 156.44it/s]
Loss:-0.80
Epoch 5: 100% 240/240 [00:01<00:00, 160.78it/s]
Loss:-0.83
Epoch 6: 100% 240/240 [00:01<00:00, 162.42it/s]
Loss:-0.86
Epoch 7: 100% 240/240 [00:01<00:00, 166.38it/s]
Loss:-0.87
Epoch 8: 100% 240/240 [00:01<00:00, 163.81it/s]
Loss:-0.86
Epoch 9: 100% 240/240 [00:01<00:00, 162.21it/s]
Loss:-0.86
Epoch 10: 100% 240/240 [00:01<00:00, 159.08it/s]
Loss:-0.88
Epoch 11: 100% 240/240 [00:01<00:00, 159.74it/s]
Loss:-0.88
Epoch 12: 100% 240/240 [00:01<00:00, 165.40it/s]
Loss:-0.88
Epoch 

After training and teaching, now we can test **teacher**, **student** and **noKD**.
We also quantize the teacher using dynamic quantization *(The quantization is taken after the model is trained)*

In [ ]:
!python test.py

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Result of testing teacher model
false positive (FP): 665, false negative (FN): 34, true positive (TP): 9966, true negative (TN): 489335
Test set: Average loss: 0.0002, Accuracy: 99.86%). Total time = 381.48191475868225
Precision: 93.745%, Recall: 99.660%, F1-measure: 96.612%
Result of testing student model
false positive (FP): 725, false negative (FN): 15, true positive (TP): 9985, true negative (TN): 489275
Test set: Average loss: 0.0005, Accuracy: 99.85%). Total time = 123.48138689994812
Precision: 93.231%, Recall: 99.850%, F1-measure: 96.427%
Result of testing noKD model
false positive (FP): 1581, false negative (FN): 645, true positive (TP): 9355, true negative (TN): 488419
Test set: Average loss: 0.0008

At here, we also demo **prunning** technique.


In [ ]:
!python prune.py

Iterative Pruning + Fine-Tuning...
Pruning and Finetuning 1/40
Pruning...
Fine-tuning...
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:943: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:926.)
  self.dropout, self.training, self.bidirectional, self.batch_first)
Layer (type:depth-idx)                   Output Shape              Param #
DistilLog                                [50, 2]                   --
├─GRU: 1-1                               [50, 50, 128]             136,433
├─Linear: 1-2                            [50, 2]                   220
Total params: 136,653
Trainable params: 136,653
Non-trainable params: 0
Total mult-adds (M): 341.09
Input size (MB): 0.30
Forward/backward pass size (MB): 2.56
Params size (MB): 0.55
E